In [44]:
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import os
import folium
import folium.plugins as plugins
from folium.features import DivIcon
from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import mpld3
import branca
import base64


%store -r file_name

met_data = pd.read_csv(file_name)
station_data = pd.read_csv('station_info_new.csv').rename(columns={'id':'station'})

merged_data = pd.merge(met_data, station_data, on='station')

# calculate the dewpoint and add the dataframe
td = dewpoint_from_relative_humidity(merged_data['tl'].values * units.degC, merged_data['rf'].values * units.percent)

merged_data['td'] = td.magnitude.round(decimals=1)
merged_data['ffxkmh'] = (merged_data['ffx'] * 3.6).round(decimals=1)
merged_data['time'] = pd.to_datetime(merged_data['time'])

# extract = merged_data[merged_data['name'].str.contains(r'Mariazell|Lilienfeld|St.P')][['tl','time','name', 'lat', 'lon']]
extract = merged_data[['tl','tlmax','td','dd', 'time','ffxkmh', 'ddx', 'name', 'lat', 'lon', 'altitude']]


# # Plot the responses for different events and regions
# sns.lineplot(x="time", y="tl",
#              hue="name",
#              data=extract)

# sns.despine()




In [45]:
# extract.loc[extract['time'] == extract['time'].iloc[-4]].dropna()

# This creates the times that are in the csv file
indexed = extract.set_index('time').drop_duplicates().sort_values(by=['name', 'time']).dropna()
individual_dates = indexed.index.unique()
latest = indexed.loc[indexed.index == individual_dates[-2]]



# this yields the values for the last time stamp
# latest = indexed.loc[indexed.index == individual_dates[-3]].dropna()
# individual_dates

latest






,tl,tlmax,td,dd,ffxkmh,ddx,name,lat,lon,altitude
time,,,,,,,,,,
2024-09-09 15:30:00+00:00,13.4,13.4,11.8,178.0,7.2,195.0,Abtenau,47.566670,13.343330,709.0
2024-09-09 15:30:00+00:00,13.6,13.6,12.5,282.0,8.6,256.0,Achenkirch,47.499440,11.706670,931.0
2024-09-09 15:30:00+00:00,13.4,13.6,11.3,20.0,4.7,5.0,Aflenz,47.545940,15.240690,783.2
2024-09-09 15:30:00+00:00,14.0,14.0,10.2,231.0,11.2,234.0,Aigen im Ennstal,47.532780,14.138260,641.0
2024-09-09 15:30:00+00:00,14.3,14.7,12.7,236.0,17.3,195.0,Alberschwende,47.457500,9.848610,715.3
...,...,...,...,...,...,...,...,...,...,...
2024-09-09 15:30:00+00:00,16.7,16.8,15.6,294.0,15.1,306.0,Wolkersdorf,48.388056,16.508889,174.0
2024-09-09 15:30:00+00:00,14.9,14.9,14.3,34.0,11.2,31.0,Wörterberg,47.226190,16.096780,403.6
2024-09-09 15:30:00+00:00,13.7,13.8,10.3,123.0,13.0,114.0,Zell am See,47.290000,12.786390,754.0


In [46]:
def create_temp_plot(data, extreme_values=True, var='tl'):


    test = data.sort_values(by=['name', 'time']).dropna()

    names_unique = test['name'].unique().tolist()
    
    fig_list, max_list, min_list = [], [], []
    lat_list, lon_list = [], []
    
    
    for name in names_unique:
        data = test.loc[test['name'] == name]
        
        

        lat_list.append(data['lat'].unique()[0])
        lon_list.append(data['lon'].unique()[0])

        fig, axs = plt.subplots(nrows=1, ncols=1)
        sns.lineplot(data=data, x='time', y=var, ax=axs, lw=4)
        sns.despine()
        fig.autofmt_xdate()
        axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

        axs.set_xlabel('Time (UTC)', fontsize=16)
        axs.set_ylabel('Temperature', fontsize=16)
        # plt.title(name)

        if extreme_values == True:
            # Find max value and index of max value along columns
            max = data.max()
            idxmax = data.idxmax()
            # Find min value and index of max value along columns
            min = data.min()
            idxmin = data.idxmin()



            axs.scatter(idxmax[var], max[var], s=100, marker='o', lw=1, zorder=3, facecolors='none', edgecolors='r')
            axs.scatter(idxmin[var], min[var], s=100, marker='o', lw=1, zorder=3, facecolors='none', edgecolors='r')



            axs.set_ylim([min[var] -2, max[var] + 2])
            
            axs.text(idxmax[var] - datetime.timedelta(hours=1.2), max[var] + 0.6, str(max[var]), fontsize=18, alpha=0.7)
            axs.text(idxmin[var] - datetime.timedelta(hours=1.5), min[var]-1.7, str(min[var]), fontsize=18, alpha=0.7)
            
            
            folder = "C:\\Users\\sh16450\\OneDrive - University of Bristol\\Documents\\GitHub\\openweatherviz\\folium\\content\\"
            
            save_str = folder + name + '_' + var + '.png'
            
            fig.savefig(save_str)
            


            plt.close(fig)
            
            fig_list.append(save_str)
            min_list.append(min[var])
            max_list.append(max[var])
            
            print(name)
            

            
            

            
            
            
        
    return fig_list, names_unique, min_list, max_list, lat_list, lon_list

fig_list, names, min_list, max_list, lat_list, lon_list = create_temp_plot(indexed, extreme_values=True, var='tl')








Abtenau
Achenkirch
Aflenz
Aigen im Ennstal
Alberschwende
Allentsteig
Alpbach
Altmünster
Amstetten
Andau
Arriach
Aspach
Aspang
Bad Aussee
Bad Bleiberg
Bad Deutsch-Altenburg
Bad Eisenkappel
Bad Gastein
Bad Gleichenberg
Bad Goisern
Bad Ischl
Bad Mitterndorf
Bad Radkersburg
Bad Tatzmannsdorf
Bad Vöslau
Bad Zell
Berndorf
Bernstein
Bischofshofen
Bludenz
Brand
Bregenz
Brenner
Bruck an der Mur
Bruckneudorf
Brunn am Gebirge
Brunnenkogel
Buchberg
Bärnkopf
Dellach
Deutschlandsberg
Dornbirn
Döllach
Ehrwald
Eisenstadt Nordost
Enns
Feistritz ob Bleiburg
Feldbach
Feldkirch
Feldkirchen
Ferlach
Feuerkogel
Fischbach
Flattnitz
Fraxern
Freistadt
Fresach
Friesach
Frohnleiten
Fürstenfeld
Gallspach
Galtür
Galzig
Gars am Kamp
Gaschurn
Gleisdorf
Gmünd
Golling
Graz Straßgang
Groß-Enzersdorf
Großarltal
Gröbming
Gumpoldskirchen
Gutenstein Mariahilfberg
Gänserndorf Stadt
Güssing
Hahnenkamm-Ehrenbachhöhe
Haiming
Hall bei Admont
Hartberg
Hermagor
Hintertux
Hochfilzen
Hohe Wand Hochkogelhaus
Hohe Wand-Hochkogelhaus
H

In [83]:
dict = {'name': names, 'tl_figure': fig_list, 'tl_min': min_list, 'tl_max': max_list} 
df_extremes = pd.DataFrame(dict)
df_extremes = df_extremes.merge(latest, on='name')


def get_popup(fig_loc, name_str, elev_str, tl_max_str, tl_min_str):
    encoded = base64.b64encode(open(fig_loc, 'rb').read()).decode()
    html = f'''
        <h1>Station: {name_str}</h1>
        <p style="font-size:14px; color:#000000; font-weight:bold;">
        Elevation: {elev_str} m
        <br />
        <span style="color:red">Tmax: {tl_max_str}&deg;C<span/>
        <br />
        <span style="color:blue">Tmin: {tl_min_str}&deg;C<span/>
        </p>
        <img src="data:image/png;base64,{encoded}">'''
    iframe = folium.IFrame(html, width=650, height=650)
    popup = folium.Popup(iframe)
    
    return popup, html

# get the popups for tl
popup_list, html_list = [], []
for i in range(np.shape(latest)[0]):
    popup, html = get_popup(df_extremes['tl_figure'][i], latest['name'].iloc[i], latest['altitude'].iloc[i],df_extremes['tl_max'][i],df_extremes['tl_min'][i])
    popup_list.append(popup)
    html_list.append(html)


df_extremes

,name,tl_figure,tl_min,tl_max,tl,tlmax,td,dd,ffxkmh,ddx,lat,lon,altitude
0,Abtenau,C:\Users\sh16450\OneDrive - University of Bris...,12.7,23.7,13.4,13.4,11.8,178.0,7.2,195.0,47.566670,13.343330,709.0
1,Achenkirch,C:\Users\sh16450\OneDrive - University of Bris...,11.4,18.1,13.6,13.6,12.5,282.0,8.6,256.0,47.499440,11.706670,931.0
2,Aflenz,C:\Users\sh16450\OneDrive - University of Bris...,12.6,23.1,13.4,13.6,11.3,20.0,4.7,5.0,47.545940,15.240690,783.2
3,Aigen im Ennstal,C:\Users\sh16450\OneDrive - University of Bris...,12.7,24.4,14.0,14.0,10.2,231.0,11.2,234.0,47.532780,14.138260,641.0
4,Alberschwende,C:\Users\sh16450\OneDrive - University of Bris...,11.9,17.2,14.3,14.7,12.7,236.0,17.3,195.0,47.457500,9.848610,715.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Wolkersdorf,C:\Users\sh16450\OneDrive - University of Bris...,15.7,28.7,16.7,16.8,15.6,294.0,15.1,306.0,48.388056,16.508889,174.0
266,Wörterberg,C:\Users\sh16450\OneDrive - University of Bris...,14.1,27.0,14.9,14.9,14.3,34.0,11.2,31.0,47.226190,16.096780,403.6
267,Zell am See,C:\Users\sh16450\OneDrive - University of Bris...,11.6,21.4,13.7,13.8,10.3,123.0,13.0,114.0,47.290000,12.786390,754.0
268,Zeltweg,C:\Users\sh16450\OneDrive - University of Bris...,13.6,24.8,15.9,15.9,11.7,74.0,7.2,54.0,47.199830,14.746890,677.6


In [85]:


m = folium.Map(location=[47.73, 13.48], zoom_start=8, tiles=None)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=MglwGpuT1VUaVc2LSRX042X3krQz6y2cOZfGtNqWg3VMUke6gemUIWz7G1uxPPXP',
                 attr='CartoDB.Voyager', max_zoom=20).add_to(base_map)
base_map.add_to(m)

# Add the date as a title to the map
map_title = individual_dates[-2].strftime("%Y-%m-%d   %H:%M UTC")
title_html = f'<h1 style="position:absolute;z-index:100000;left:10vw;background-color:#eeeee4;" >{map_title}</h1>'
m.get_root().html.add_child(folium.Element(title_html))


COLORS = [
    (lambda x: x > 40, "#EC8989"),
    (lambda x: 35 < x <= 40, "#EA3D09"),
    (lambda x: 30 < x <= 35, "#EA3D09"),
    (lambda x: 25 < x <= 30, "#EA6509"),
    (lambda x: 20 < x <= 25, "#C5C71B"),
    (lambda x: 15 < x <= 20, "#8EF700"),
    (lambda x: 10 < x <= 15, "#07AF02"),
    (lambda x: 5 < x <= 10, "#94EA7F"),
    (lambda x: 0 < x <= 5, "#9cecc8"),
    (lambda x: 0 >= x >= -5, "#18B5DA"),
    (lambda x: -5 > x >= -10, "#1661e8"),
    (lambda x: -10 > x >= -15, "#A375E6"),
    (lambda x: -15 > x >= -20, "#ce8bf0"),
    (lambda x: x < -20, "#EC9CE6"),
]

COLORS_WIND = [
    (lambda x: x >= 100, "#EF06E1"), # purple pizazz
    (lambda x: 80 <= x < 100, "#EF0620"), # torch red
    (lambda x: 60 <= x < 80, "#FB5D04"), # blaze orange
    (lambda x: 40 <= x < 60, "#D8DC0F"), # bitter lemon
    (lambda x: 30 <= x < 40, "#4FBF2A"), # apple
    (lambda x: x < 30, "#000000"), # black
    
    
]



# This picks the color for temperature and tmax, dewpoint and wind
def get_color(x, color_list):
    for predicate, color in color_list:
        if predicate(x):
            return color

    return 'black'



# Add analysis for temperature
fg = folium.FeatureGroup(name="Temperature", control=True, overlay=False).add_to(m)
# marker_cluster = plugins.MarkerCluster(options={'disableClusteringAtZoom':10, 'maxClusterRadius':5, 'singleMarkerMode':False}).add_to(m)

for i in range(np.shape(latest)[0]):
        color = get_color(latest['tl'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(latest['tl'].iloc[i]) + '</div>'
        p1 = [latest['lat'].iloc[i], latest['lon'].iloc[i]]
        folium.Marker(p1,popup=popup_list[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fg)

        # fg.add_child(popup_list[i])
 
 
# Add analysis for maximum temperature        
fgg = folium.FeatureGroup(name="Tmax", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
        color = get_color(latest['tlmax'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(latest['tlmax'].iloc[i]) + '</div>'
        p1 = [latest['lat'].iloc[i], latest['lon'].iloc[i]]
        folium.Marker(p1,popup=latest['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgg)
        


# Add analysis for dewpoint temperature        
fggg = folium.FeatureGroup(name="Dewpoint", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
        color = get_color(latest['td'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(latest['td'].iloc[i]) + '</div>'
        p1 = [latest['lat'].iloc[i], latest['lon'].iloc[i]]
        folium.Marker(p1,popup=latest['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fggg)
        
# Add analysis for 24h maximum temperature        
tm = folium.FeatureGroup(name="24h Maximum Temperature", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(df_extremes)[0]):
        color = get_color(df_extremes['tl_max'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(df_extremes['tl_max'].iloc[i]) + '</div>'
        p1 = [df_extremes['lat'].iloc[i], df_extremes['lon'].iloc[i]]
        folium.Marker(p1,popup=df_extremes['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(tm)
        
# Add analysis for 24h maximum temperature        
tmm = folium.FeatureGroup(name="24h Minmum Temperature", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(df_extremes)[0]):
        color = get_color(df_extremes['tl_min'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(df_extremes['tl_min'].iloc[i]) + '</div>'
        p1 = [df_extremes['lat'].iloc[i], df_extremes['lon'].iloc[i]]
        folium.Marker(p1,popup=df_extremes['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(tmm)
        



# Add analysis for wind direction of max gust and gust direction
ddg = folium.FeatureGroup(name="Wind direction (max gust) + Max Gust (kmh)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(latest)[0]):
    rotate_str = str(latest['ddx'].iloc[i] + 90)
    color = get_color(latest['ffxkmh'].iloc[i], COLORS_WIND)
    folium.Marker(location=[latest['lat'].iloc[i], latest['lon'].iloc[i]], popup=latest['name'].iloc[i],
                            icon=plugins.BeautifyIcon(icon="fa-solid fa-arrow-right",
                                            border_color='transparent',
                                            background_color='transparent',
                                            border_width=1,
                                            text_color=color,
                                            inner_icon_style='margin:0px;font-size:24px;transform: rotate({0}deg);'.format(rotate_str))).add_to(ddg)

folium.LayerControl().add_to(m)

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


# icon = folium.DivIcon(html="17.4")
# folium.Marker([47.0, 15.0], icon=icon).add_to(m)
m.save(os.path.join('simple_popups.html'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27